# Solar Potential Analysis

*Calculates rooftop solar potential, economic feasibility, and payback periods using DSM, building footprints, and zoning data. Includes Plane of Array (POA) correction for tilt and azimuth.*

In [1]:
!pip install rasterstats

In [7]:
import os
import numpy as np
import rasterio
import geopandas as gpd
import pandas as pd
from rasterio.warp import calculate_default_transform, reproject, Resampling
from matplotlib.colors import LightSource

# Section 1: Configuration & File Paths

# Input: Digital Surface Model (DSM) Raster
DSM_PATH = 'inputs/download/University_City_Height.tif'

# Input: Processed Building Footprints (Vector)
BUILDINGS_PATH = 'inputs/processed/University_City_Buildings_with_Height.geojson'

# Input: Zoning Boundaries
ZONING_PATH = 'inputs/boundaries/philly_zoning.geojson'

# Output Configuration
OUTPUT_DIR = 'outputs/final_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Intermediate Output Paths
DSM_PROJ_PATH = os.path.join(OUTPUT_DIR, 'dsm_projected.tif')
SLOPE_PATH = os.path.join(OUTPUT_DIR, 'slope.tif')
ASPECT_PATH = os.path.join(OUTPUT_DIR, 'aspect.tif')
SHADE_PATH = os.path.join(OUTPUT_DIR, 'shading_factor.tif')
FINAL_GEOJSON = os.path.join(OUTPUT_DIR, 'rooftop_solar_potential_optimized.geojson')


# Section 2: Physical & Economic Parameters

GLOBAL_HORIZONTAL_IRRADIANCE = 1469  # kWh/m2/year
PANEL_EFFICIENCY = 0.165             # 16.5% efficiency
PERFORMANCE_RATIO = 0.75             # System losses (inverter, wiring, soiling)

# Packing Factor / Installation Factor
# 0.7 implies only 70% of the roof area is usable for panels. This accounts for edge setbacks, HVAC obstructions, and maintenance pathways.
PACKING_FACTOR = 0.7

# Installation Costs ($/Watt)
INSTALL_COST_PER_WATT_RESIDENTIAL = 3.25
INSTALL_COST_PER_WATT_COMMERCIAL = 1.55
INSTALL_COST_PER_WATT_INDUSTRIAL = 1.55
INSTALL_COST_PER_WATT_OTHER = 1.55
INSTALL_COST_PER_WATT_UNKNOWN = 3.25

# Incentives
FED_TAX_CREDIT_RATE = 0.30  # Federal ITC

# Electricity Rates ($/kWh) - Pennsylvania Averages
ELEC_RATE_PENNSYLVANIA_RESIDENTIAL = 0.1777
ELEC_RATE_PENNSYLVANIA_COMMERCIAL = 0.1103
ELEC_RATE_PENNSYLVANIA_INDUSTRIAL = 0.0787
ELEC_RATE_PENNSYLVANIA_OTHER = 0.1103
ELEC_RATE_PENNSYLVANIA_UNKNOWN = 0.1777


# Section 3: Core Helper Functions

def reproject_dsm(input_path, output_path, target_crs='EPSG:26918'):
    """
    Reproject DSM to a projected CRS.
    Essential for accurate slope calculation (meters vs degrees).
    """
    print(f"Reprojecting DSM to {target_crs} ...")
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({'crs': target_crs, 'transform': transform, 'width': width, 'height': height})

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.bilinear)
    return output_path

def calculate_slope_aspect(dsm_path, slope_out, aspect_out):
    """
    Calculate topographic parameters: Slope and Aspect.
    """
    print("Calculating topographic parameters (Slope & Aspect)...")
    with rasterio.open(dsm_path) as src:
        dsm = src.read(1)
        # Calculate cell size to ensure correct gradient units
        cell_size = (src.transform[0] + abs(src.transform[4])) / 2
        
        dy, dx = np.gradient(dsm, cell_size)
        
        slope_deg = np.degrees(np.arctan(np.sqrt(dx*dx + dy*dy)))
        aspect_deg = np.degrees(np.arctan2(-dx, dy))
        aspect_deg = np.where(aspect_deg < 0, aspect_deg + 360, aspect_deg)
        
        meta = src.meta.copy()
        meta.update(dtype=rasterio.float32)
        
        with rasterio.open(slope_out, 'w', **meta) as dst:
            dst.write(slope_deg.astype(rasterio.float32), 1)
        with rasterio.open(aspect_out, 'w', **meta) as dst:
            dst.write(aspect_deg.astype(rasterio.float32), 1)
            
    return slope_deg, aspect_deg

def calculate_occlusion_factor(dsm_path, gdf_roofs, output_tif_path=SHADE_PATH):
    """
    Simulate shading using 9-Angle Hillshade Analysis.
    Averages illumination from multiple solar positions throughout the year.
    """
    print("Simulating solar occlusion (9-Angle Analysis)...")
    
    with rasterio.open(dsm_path) as src:
        dsm = src.read(1)
        # Handle nodata
        if src.nodata is not None:
            dsm[dsm == src.nodata] = np.nan
        
        # Solar positions (Azimuth, Altitude)
        solar_moments = [
            (135, 35), (180, 45), (225, 35), # Equinox (Spring/Autumn)
            (105, 60), (180, 70), (255, 60), # Summer Solstice
            (150, 15), (180, 25), (210, 15)  # Winter Solstice
        ]
        
        cumulative_shade = np.zeros_like(dsm, dtype=np.float32)
        
        for i, (az, alt) in enumerate(solar_moments):
            ls = LightSource(azdeg=az, altdeg=alt)
            hs = ls.hillshade(dsm, vert_exag=1, dx=src.res[0], dy=src.res[1])
            cumulative_shade += hs

        avg_shade = cumulative_shade / len(solar_moments)
        
        # Save shading raster
        if output_tif_path:
            meta = src.meta.copy()
            meta.update(dtype=rasterio.float32, count=1, nodata=None)
            with rasterio.open(output_tif_path, 'w', **meta) as dst:
                dst.write(avg_shade.astype(rasterio.float32), 1)
        
        # Sample raster values to vector centroids
        print("   -> Sampling shading values to roof centroids...")
        shading_factors = []
        for point in gdf_roofs.geometry.centroid:
            try:
                row, col = src.index(point.x, point.y)
                if 0 <= row < avg_shade.shape[0] and 0 <= col < avg_shade.shape[1]:
                    val = avg_shade[row, col]
                    shading_factors.append(1.0 if np.isnan(val) else float(val))
                else:
                    shading_factors.append(1.0)
            except:
                shading_factors.append(1.0)
                
    return shading_factors

def sample_raster_attributes(gdf, raster_path, col_name):
    """
    Generic function to sample raster values at vector geometries.
    """
    print(f"Sampling attribute: {col_name}")
    with rasterio.open(raster_path) as src:
        # Use generator to optimize memory usage
        coords = [(g.representative_point().x, g.representative_point().y) for g in gdf.geometry]
        sampled_values = [x[0] for x in src.sample(coords)]
        gdf[col_name] = sampled_values
    return gdf


# Section 4: Filtering & Economic Analysis

def smart_filter(gdf):
    """Apply suitability filtering rules based on geometry and orientation."""
    print("Applying smart filters...")
    # Area Threshold
    gdf['area_m2'] = gdf.geometry.area
    valid_area = gdf['area_m2'] > 5
    
    # Flat vs. Tilted Roof Logic
    is_flat = gdf['mean_slope'] < 10
    # South-facing logic: Aspect between 90 (East) and 270 (West)
    is_tilted_south = (gdf['mean_slope'] >= 10) & (gdf['mean_aspect'] > 90) & (gdf['mean_aspect'] < 270)
    
    suitable = gdf[valid_area & (is_flat | is_tilted_south)].copy()
    print(f"   - Original Count: {len(gdf)} -> After Filtering: {len(suitable)}")
    return suitable

def enrich_with_economics_and_zoning(gdf):
    """Match zoning data and calculate financial metrics."""
    print("Calculating economic indicators...")
    
    # Step 1: Zoning Data Matching
    gdf['zoning_type'] = 'Unknown' # Default value
    
    if os.path.exists(ZONING_PATH):
        try:
            print(f"   -> Loading Zoning File: {os.path.basename(ZONING_PATH)}")
            zoning_gdf = gpd.read_file(ZONING_PATH)
            
            # Align Coordinate Reference Systems (CRS)
            if zoning_gdf.crs != gdf.crs: 
                zoning_gdf = zoning_gdf.to_crs(gdf.crs)
            
            # Target Column for Zoning Codes
            target_col = 'long_code'
            
            if target_col in zoning_gdf.columns:
                # Spatial Join (Keep necessary columns only)
                gdf = gpd.sjoin(gdf, zoning_gdf[[target_col, 'geometry']], how='left', predicate='intersects')
                
                # Remove duplicates: One roof should match one zone
                gdf = gdf[~gdf.index.duplicated(keep='first')]
                
                # Define Mapping Logic (Philadelphia Standard)
                def map_zone(val):
                    if pd.isna(val): return 'Other'
                    s = str(val).strip().upper()
                    if s.startswith('R'): return 'Residential'
                    if s.startswith('C'): return 'Commercial'
                    if s.startswith('I'): return 'Industrial'
                    return 'Other'
                
                # Apply Mapping
                gdf['zoning_type'] = gdf[target_col].apply(map_zone)
                
                # Print Statistics
                counts = gdf['zoning_type'].value_counts()
                print(f"   Zoning match successful! Stats: {counts.to_dict()}")
            else:
                print(f"   Error: Column '{target_col}' not found in zoning file.")

        except Exception as e:
            print(f"   Zoning match error: {e}")
    else:
        print("   Zoning file not found. Skipping zoning match.")

    # Step 2: Assign Economic Parameters
    print("   -> Assigning costs and electricity rates...")
    conditions = [
        gdf['zoning_type'] == 'Residential',
        gdf['zoning_type'] == 'Commercial',
        gdf['zoning_type'] == 'Industrial',
        gdf['zoning_type'] == 'Other'
    ]
    
    gdf['install_cost_per_watt'] = np.select(conditions, 
        [INSTALL_COST_PER_WATT_RESIDENTIAL, INSTALL_COST_PER_WATT_COMMERCIAL, 
         INSTALL_COST_PER_WATT_INDUSTRIAL, INSTALL_COST_PER_WATT_OTHER], 
        default=INSTALL_COST_PER_WATT_UNKNOWN)
        
    gdf['elec_rate'] = np.select(conditions, 
        [ELEC_RATE_PENNSYLVANIA_RESIDENTIAL, ELEC_RATE_PENNSYLVANIA_COMMERCIAL, 
         ELEC_RATE_PENNSYLVANIA_INDUSTRIAL, ELEC_RATE_PENNSYLVANIA_OTHER], 
        default=ELEC_RATE_PENNSYLVANIA_UNKNOWN)

    # Step 3: Core Financial Calculations
    if 'shading_factor' not in gdf.columns: gdf['shading_factor'] = 0.95
    
    # Annual Potential Generation (kWh)
    gdf['potential_kwh'] = (gdf['area_m2'] * PACKING_FACTOR * PANEL_EFFICIENCY * GLOBAL_HORIZONTAL_IRRADIANCE * PERFORMANCE_RATIO * gdf['shading_factor'])
                            
    # System Size and Cost
    gdf['system_size_kw'] = gdf['area_m2'] * PACKING_FACTOR * PANEL_EFFICIENCY
    gdf['initial_cost'] = gdf['system_size_kw'] * 1000 * gdf['install_cost_per_watt']
    gdf['net_cost'] = gdf['initial_cost'] * (1 - FED_TAX_CREDIT_RATE)
    
    # Financial Returns
    gdf['yearly_savings'] = gdf['potential_kwh'] * gdf['elec_rate']
    
    # Payback Period (Years)
    gdf['payback_years'] = gdf.apply(
        lambda x: x['net_cost'] / x['yearly_savings'] if x['yearly_savings'] > 10 else 99, axis=1)
        
    return gdf


# Section 5: Main Execution Flow

print("Starting Solar Potential Analysis (based on GlobalBuildingAtlas)...")

if os.path.exists(DSM_PATH) and os.path.exists(BUILDINGS_PATH):
    # 1. Reproject DSM
    dsm_proj = reproject_dsm(DSM_PATH, DSM_PROJ_PATH)
    
    # 2. Calculate Topographic Factors
    calculate_slope_aspect(dsm_proj, SLOPE_PATH, ASPECT_PATH)
    
    # 3. Load and Transform Vector Data
    print("Loading building footprints...")
    gdf = gpd.read_file(BUILDINGS_PATH)
    
    # Ensure vector CRS matches projected raster (EPSG:26918)
    if gdf.crs.to_string() != 'EPSG:26918':
        print(f"   -> Transforming CRS: {gdf.crs} -> EPSG:26918")
        gdf = gdf.to_crs('EPSG:26918')
        
    # 4. Sample Raster Attributes to Vector
    gdf = sample_raster_attributes(gdf, SLOPE_PATH, 'mean_slope')
    gdf = sample_raster_attributes(gdf, ASPECT_PATH, 'mean_aspect')
    
    # 5. Calculate Shading Factors
    gdf['shading_factor'] = calculate_occlusion_factor(dsm_proj, gdf)
    
    # 6. Filtering & Economic Calculation
    gdf = gdf[gdf['shading_factor'] > 0.4] # Preliminary filter for severe shading
    gdf_filtered = smart_filter(gdf)
    final_gdf = enrich_with_economics_and_zoning(gdf_filtered)
    
    # 7. Save Results (Convert back to WGS84 for Web Mapping)
    final_gdf = final_gdf.to_crs("EPSG:4326")
    final_gdf.to_file(FINAL_GEOJSON, driver='GeoJSON')
    
    print(f"\nAnalysis Complete! Results saved to: {FINAL_GEOJSON}")
    print(f"   - Total Annual Potential: {final_gdf['potential_kwh'].sum()/1000:,.2f} MWh")
    print(f"   - Median Payback Period: {final_gdf['payback_years'].median():.1f} Years")

else:
    print(f"Missing required files:\n  DSM: {DSM_PATH}\n  Buildings: {BUILDINGS_PATH}")

Starting Solar Potential Analysis (based on GlobalBuildingAtlas)...
Reprojecting DSM to EPSG:26918 ...
Calculating topographic parameters (Slope & Aspect)...
Loading building footprints...
   -> Transforming CRS: EPSG:4326 -> EPSG:26918
Sampling attribute: mean_slope
Sampling attribute: mean_aspect
Simulating solar occlusion (9-Angle Analysis)...
   -> Sampling shading values to roof centroids...
Applying smart filters...
   - Original Count: 7697 -> After Filtering: 4608
Calculating economic indicators...
   -> Loading Zoning File: philly_zoning.geojson
   Zoning match successful! Stats: {'Residential': 3774, 'Commercial': 604, 'Other': 186, 'Industrial': 44}
   -> Assigning costs and electricity rates...

Analysis Complete! Results saved to: outputs/final_results\rooftop_solar_potential_optimized.geojson
   - Total Annual Potential: 133,228.62 MWh
   - Median Payback Period: 14.6 Years
